# **Import Libarary**

In [127]:
# Manipulasi Data
import pandas as pd 
import numpy as np
import re
import string
import emoji

# Visualisasi Data
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# NLP & Preprocessing
import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Machine Learning & Sentiment Analysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier


In [128]:
# Download NLTK Resources
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/bagasnuryaman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/bagasnuryaman/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bagasnuryaman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bagasnuryaman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# **Loading Data**

In [129]:
# Membaca dataset
df = pd.read_csv('ff_scraping.csv')

# Menampilkan nya
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,169fd249-9e14-45b3-9fa5-b004f8792271,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"sedih bgt njir, akun aku hilang ud level 40an",2,0,1.109.1,2025-05-20 21:30:40,NaN,NaN,1.109.1
1,a59942c7-c9b3-4ffd-a026-3589cbb5be3c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,game bayak buk bintang 1 Update terus buk nya ...,1,1,1.109.1,2025-05-20 21:29:45,NaN,NaN,1.109.1
2,1648cc00-59f2-4dcf-ac05-b974911f3aba,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,baguss gamenya,5,0,NaN,2025-05-20 21:22:48,NaN,NaN,NaN
3,b3e7444a-f50c-415a-b030-2eff43d394c9,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,ehh min lu klo ngasih tim yg ngotak apaa masa ...,1,0,1.109.1,2025-05-20 21:22:35,NaN,NaN,1.109.1
4,66863758-112b-4c94-b583-47b7306f0a6c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,bagus banget game nya,5,0,NaN,2025-05-20 21:19:35,NaN,NaN,NaN


# **Data Preprocessing**

In [130]:
# Menampilkan informasi dari dataset 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              10000 non-null  object
 1   userName              10000 non-null  object
 2   userImage             10000 non-null  object
 3   content               10000 non-null  object
 4   score                 10000 non-null  int64 
 5   thumbsUpCount         10000 non-null  int64 
 6   reviewCreatedVersion  6508 non-null   object
 7   at                    10000 non-null  object
 8   replyContent          7 non-null      object
 9   repliedAt             7 non-null      object
 10  appVersion            6508 non-null   object
dtypes: int64(2), object(9)
memory usage: 859.5+ KB


- Bisa kita lihat ada `Missing Values` pada fitur **reviewCreatedVersion**, **replyContent**, **repliedAt**, dan juga **appVersion**. 
- Disini aku bakal mengambil fitur **Score** dan juga **Content** saja.

## **Mengambil fitur yang akan digunakan saja**

In [131]:
ff_clean = df[['score', 'content']].copy()

## **Penanganan Missing Values**

In [132]:
# Menampilkan Fitur yang missing values
missing_values = ff_clean.isnull().sum()
missing_values_percentage = (missing_values / len(ff_clean)) * 100
missing_values_ff_clean = pd.DataFrame({'Jumlah Missing Values': missing_values, 'Persentase': missing_values_percentage})
print(f'Keseluruhan Missing Values dari dataset ff_clean ini adalah {ff_clean.isnull().sum().sum()}')

# Cek hasil
missing_values_ff_clean

Keseluruhan Missing Values dari dataset ff_clean ini adalah 0


,Jumlah Missing Values,Persentase
score,0,0.0
content,0,0.0


- Soleh Juga gak ada missing values nya.

## **Penanganan Duplikasi Data**

In [133]:
print(f'Jumlah data yang terkena duplikasi data adalah : {ff_clean.duplicated().sum()}')

Jumlah data yang terkena duplikasi data adalah : 1776


- Disini akan aku hapus saja.

In [134]:
ff_clean.drop_duplicates(inplace=True)
print(f'Jumlah data yang terkena duplikasi data adalah : {ff_clean.duplicated().sum()}')

Jumlah data yang terkena duplikasi data adalah : 0


## **Preprocessing Text**

### **Cleaning Text**

In [135]:
# ----- 1. Cleaning Text ------
def clean_text(text):
    """
    Fungsi untuk membersihkan teks dari karakter karakter yang tidak di inginkan seperti angka, simbol, emoji, url, karakter khusus dll.
    """
    text = re.sub(r'@\w+', '', text) # Menghapus mention
    text = re.sub(r'#(\w+)', r'\1', text) # Menghapus hashtag
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Menghapus URL
    text = re.sub(r'\bRT\b', '', text) # Menghapus RT
    text = re.sub(r'\d+', '', text) # Menghapus angka
    text = re.sub(r'\s+', ' ', text) # Menghapus whitespace berlebih
    
    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # Menghapus tanda baca
    text = text.strip() # Menghapus whitespace di awal dan akhir

    # Menghapus emoji
    text = emoji.replace_emoji(text, replace='')
    return text

- Langkah awal untuk `Preprocessing Text` adalah dengan cara membersihkan text dari karakter karakter yang tidak di inginkan seperti angka, emoji, url, simbol, karakter khusus, tanda baca, dll. Makanya itu aku membuat fungsi **`clean_text`**.

### **Case Folding**

In [136]:
# ----- 2. Case Folding -----
def case_folding(text):
    """
    Fungsi untuk mengubah teks menjadi huruf kecil
    """
    return text.lower()

- Langkah keduanya yaitu membuat text atau kalimat kalimat menjadi huruf kecil, karena dalam NLP ini di haruskan banget seragam, maka dari itu aku membuat fungsi **`case_folding`**.

### **Slang Words**

In [137]:
# ----- 3. Normalisasi Slang Words -----
def normalize_slang(text):
    """
    Fungsi untuk menormalkan kata kata slang/informal menjadi kata baku
    """
    # Kamus slang words untuk Free Fire
    slang_dict = {
        # Game-specific
        'ff': 'free fire', 'epep': 'free fire', 'dm': 'diamond', 'daimen': 'diamond', 'daymen': 'diamond', 'daimon': 'diamond', 'gem': 'game', 'geme': 'game', 'gm': 'game', 'boyah': 'menang', 'mabar': 'main bareng', 'cs': 'clash squad', 'br': 'battle royale', 'citer': 'cheater', 'chiter': 'cheater', 'bewan': 'bermain', 'bundel': 'bundle', 'bandle': 'bundle', 'sg': 'shotgun', 'rank': 'peringkat', 'reng': 'rank', 'renk': 'rank', 'hedsot': 'headshot','hetsot': 'headshot', 'glowal': 'gloo wall', 'gllowal': 'gloo  wall', 'bag': 'bug', 'buk': 'bug', 'afk': 'meninggalkan permainan', 'gg' : 'good game', 'ggwp': 'good game well played', 'gege': 'good game', 'setep': 'step', 'sensi': 'sensitivitas kontrol', 'bot': 'pemain yang tidak handal', 'apdet': 'update', 'apdetan': 'update', 'even': 'event', 'top up': 'isi ulang diamong', 'gameplay': 'cara bermain', 'win': 'menang', 'lose': 'kalah', 'los': 'kalah', 'reload': 'isi ulang peluru', 'damage': 'kerusakan', 'kentang': 'handphone jelek', 'demek': 'damage',

        # Singkatan umum dan juga typo
        'gw': 'saya', 'gue': 'saya', 'aku': 'saya', 'lu': 'kamu', 'lo': 'kamu', 'aja': 'saja', 'aj': 'saja', 'kalo': 'kalau', 'klo': 'kalau', 'klu': 'kalau',  'bgt': 'banget', 'yg': 'yang', 'ga': 'tidak', 'gak': 'tidak', 'gk': 'tidak', 'ngga': 'tidak', 'nggak': 'tidak', 'udah': 'sudah', 'udh': 'sudah','dah': 'sudah', 'nih': 'ini', 'emg': 'memang', 'emang': 'memang', 'kyk': 'seperti', 'kyak': 'seperti', 'kek': 'seperti', 'dgn': 'dengan', 'sm': 'sama', 'tp': 'tapi', 'tpi': 'tapi', 'pake': 'pakai', 'pakek': 'pakai','ngelag': 'lag', 'nglag': 'lag', 'leg': 'lag', 'burik': 'grafiknya jelek', 'bgus': 'bagus', 'bgs': 'bagus', 'baguss': 'bagus', 'mantep': 'mantap', 'mantap': 'sangat bagus', 'keren': 'bagus', 'seru': 'menyenangkan', 'yah': 'ya', 'krn': 'karena', 'min': 'admin', 'mohon': 'minta', 'plis': 'tolong', 'pliss': 'tolong', 'tlg': 'tolong', 'jgn': 'jangan', 'bocil': 'anak kecil', 'ud': 'sudah', 'maen': 'main', 'ngasih': 'memberi', 'ngotak': 'masuk akal', 'mulu': 'terus', 'stuk': 'terjebak', 'ganiat': 'tidak niat', 'gausah': 'tidak usah', 'di bukan': 'di buka', 'ngebantu': 'membantu', 'pingin': 'ingin', 'mlah': 'malah', 'dapet': 'dapat', 'mayan': 'lumayan', 'effort': 'usaha', 'berdamage': 'berdampak', 'kasih': 'beri', 'yok': 'ayo', 'yuk': 'ayo', 'comeback': 'kembali', 'apus': 'hapus', 'kayak': 'seperti', 'jir': 'aduh', 'njir': 'aduh', 'anjir': 'aduh', 'hoki': 'keberuntungan', 'smoth': 'halus', 'smooth': 'halus', 'smpai': 'sampai', 'smp': 'sampai', 'smpe': 'sampai', 'sampe': 'sampai', 'hamdeh': 'aduh', 
    }
    
    # Membuat kata kata menjadi lowercase
    words = text.lower().split()
    
    # Normalisasi setiap kata
    normalized_words = []
    for word in words:
        # Jika kata ada dalam kamus slang, ganti dengan kata bakunya
        if word in slang_dict:
            normalized_words.append(slang_dict[word])
        else:
            normalized_words.append(word)
    
    # Gabungkan kembali kata-kata menjadi teks
    return ' '.join(normalized_words)


- Langkah ketiga adalah kita harus mengganti kata kata yang informal seperti singkatan, dan bahasa gaul menjadi kata kata yang baku, dan juga pasti ada typo nah kita juga harus membersihkan dan mengganti dengan kalimat atau kata kata yang benar, makanya dari itu aku bikin sebuah fungsi yang bernama **`normalize_slang`**.

> **ANNOUNCEMENT**

- Tapi yang perlu garis bawahi adalah kata kata typo dan slangwords ini pasti banyak, aku hanya mencoba untuk mengganti beberapa kata kata yang umum digunakan, karena keterbatasan waktu dan aku adalah manusia tidak mungkin untuk satu satu melihat review dan mengganti kata kata yang typo atau slangwordsnya.

### **Tokenization**

In [138]:
# ----- 4. Tokenization -----
def tokenization(text):
    """
    Fungsi untuk mmemecah teks atau membagi teks menjadi daftar akata atau token
    """
    token = word_tokenize(text)
    return token

- Langkah ke 4 adalah kita harus memecah teks atau kalimat menjadi daftar kata atau token yang bermakna, ini langkah yang harus di lakukan karena kalau tidak kita tidak bisa melanjutkan ke langkah berikutnya. 

### **StopWords**

In [ ]:
# ----- 5. Stopword Removal -----
def filtering_stopwords(tokens):
    """
    Fungsi untuk mengganti kata kata umum yang kurang bermakna dari daftar token
    """
    listStopWords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    custom_stopwords = [
        'iya', 'yaa', 'gak', 'nga', 'gk', 'nya', 'na', 'sih', 'ku', 'di', 'ga', 'ya', 
        'gaa', 'loh', 'kah', 'woi', 'woii', 'woy', 'dong', 'deh', 'nih', 'sih',
        'kok', 'kek', 'cuma', 'doang', 'banget', 'aja', 'doank', 'bgt', 'yg', 'udah',
        'udh', 'dah', 'tuh', 'gitu', 'gini', 'emang', 'emg', 'kan', 'kalo', 'klo',
        'yah', 'si', 'tau', 'gw', 'gue', 'ane', 'lu', 'lo', 'wkwk', 'wkwkwk', 'xixi',
        'haha', 'hehe', 'tp', 'tapi', 'karna', 'krn', 'biar', 'ehh', 'eh'
    ]

    ff_stopwords = [
    'ff', 'freefire', 'free', 'fire', 'garena', 'game', 'player', 'play', 'main',
    'maen', 'mainkan', 'bermain', 'pemain', 'char', 'character', 'karakter',
    'season', 'ranked', 'classic', 'match', 'skin', 'diamond', 'dm'
]
    listStopWords.update(ff_stopwords)
    listStopWords.update(custom_stopwords)
    listStopWords.update(listStopwords1)
    filtered = []
    for txt in tokens:
        if txt not in listStopWords:
            filtered.append(txt)
    tokens = filtered
    return tokens

- Pada tahapan ke 5 ini, kita harus mencari kata kata yang umum dan sering muncul dalam teks atau sutau kalimat tapi tidak bermakna dan tidak berkontribusi apapun di dalam hasil pemodelan. Contohnya kata **dan**, **yang**, **di** dll. Jadi intinya seperti ini aja deh, Menghapus kata kata yang tidak relevan agar analisis menjadi lebih fokus pada kata kata yang bermakna.

### **Stemming**

In [140]:
# ----- 6. Stemming -----
def stemming(tokens):
    """
    Fungsi untuk mengubah daftar kata atau token menjadi bentuk dasarnya menggunakan Sastrawi karena bahasa Indonesia
    """
    # Membuat objek stemmer 
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Melakukan stemming pada setiap token dengan list comprehension
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return stemmed_tokens

- Langkah ke 6 adalah `Stemming`, berfungsi untuk menyederhanakan kata ke bentuk dasarnya, agar model bisa memahami teks dengan lebih efisien dan akurat.

### **Joining**

In [141]:
# ----- 7. Join Tokens to Text -----
def to_sentence(tokens):
    """
    Fungsi untuk menggabungkan token menjadi kalimat
    """
    return " ".join(tokens)


- Langkah terakhir adalah menggabungkan daftar kata kata menjadi sebuah kalimat utuh lagi, karena tadi kan kita pecah pecah, dan terakhir digabungkan kembali.

### **Penerapan Function**

In [142]:
def preprocessing_text(dataset):
    """
    Menerapkan pipeline preprocessing text pada dataset dengan analisis missing values.
    """

    df = dataset.copy()
    total_data = len(df)

    print(f'Memulai preprocessing untuk {total_data} data ulasan Free Fire\n')
    
    # Fungsi untuk memeriksa dan melaporkan missing values
    def check_missing(df, column_name):
        missing = df[column_name].isna().sum()
        empty = (df[column_name].astype(str) == '').sum()
        total_invalid = missing + empty
        percent = (total_invalid / len(df)) * 100
        
        print(f'Status data: {len(df)-total_invalid} valid, {total_invalid} missing/empty ({percent:.2f}%)')
        return total_invalid

    try:
        # 1. Cleaning Text
        df['clean_text'] = df['content'].apply(clean_text)
        check_missing(df, 'clean_text')
        print(f'Proses Cleaning Text done Mamang!!!')

        # 2. Case Folding
        df['case_foldingText'] = df['clean_text'].apply(case_folding)
        check_missing(df, 'case_foldingText')
        print(f'Proses Case Folding done Mamang!!!')

        # 3. Normalisasi Slang Words 
        df['normalized_slangwordsText'] = df['case_foldingText'].apply(normalize_slang)
        check_missing(df, 'normalized_slangwordsText')
        print(f'Proses Normalisasi Slang Words done Mamang!!!')

        # 4. Tokenizing
        df['tokenized_text'] = df['normalized_slangwordsText'].apply(tokenization)

        # Untuk kolom list, periksa list kosong
        tokenize_empty = sum(1 for x in df['tokenized_text'] if not x)
        print(f'Status data: {len(df)-tokenize_empty} valid, {tokenize_empty} empty lists ({tokenize_empty/len(df)*100:.2f}%)')
        print(f'Proses Tokenizing done Mamang!!!')

        # 5. Stopword Removal
        df['stopword_removedText'] = df['tokenized_text'].apply(filtering_stopwords)
        stopword_empty = sum(1 for x in df['stopword_removedText'] if not x)
        print(f'Status data: {len(df)-stopword_empty} valid, {stopword_empty} empty lists ({stopword_empty/len(df)*100:.2f}%)')
        print(f'Proses Stopword Removal done Mamang!!!')

        # 6. Stemming
        df['stemmed_text'] = df['stopword_removedText'].apply(stemming)
        stemming_empty = sum(1 for x in df['stemmed_text'] if not x)
        print(f'Status data: {len(df)-stemming_empty} valid, {stemming_empty} empty lists ({stemming_empty/len(df)*100:.2f}%)')
        print(f'Proses Stemming done Mamang!!!')

        # 7. Join Tokens to Text
        df['final_text'] = df['stemmed_text'].apply(to_sentence)
        final_missing = check_missing(df, 'final_text')
        print(f'Proses Join Tokens to Text done Mamang!!!')

        # Ringkasan hasil preprocessing
        print('\nRINGKASAN HASIL PREPROCESSING:')
        print(f'Total data awal: {total_data}')
        print(f'Total data valid setelah preprocessing: {total_data - final_missing}')
        print(f'Total data invalid/kosong: {final_missing} ({final_missing/total_data*100:.2f}%)')
        
        # Analisis data yang hilang di tahap akhir (opsional)
        if final_missing > 0:
            print('\nAnalisis data kosong:')
            empty_indices = df[df['final_text'] == ''].index
            for idx in empty_indices[:min(3, len(empty_indices))]:
                print(f'\nData kosong #{idx}:')
                print(f'Teks asli: {df["content"].iloc[idx]}')
                print(f'Setelah cleaning: {df["clean_text"].iloc[idx]}')
                
        print(f'\nPREPROCESSING SELESAI MAMANG!')
        
    except Exception as e:
        print(f'\nWalawe Terjadi error pada preprocessing Mamang: {str(e)}')
        
    return df

In [143]:
# Implementasi Function
process_df = preprocessing_text(ff_clean)
process_df.head()

Memulai preprocessing untuk 8224 data ulasan Free Fire

Status data: 8155 valid, 69 missing/empty (0.84%)
Proses Cleaning Text done Mamang!!!
Status data: 8155 valid, 69 missing/empty (0.84%)
Proses Case Folding done Mamang!!!
Status data: 8154 valid, 70 missing/empty (0.85%)
Proses Normalisasi Slang Words done Mamang!!!
Status data: 8154 valid, 70 empty lists (0.85%)
Proses Tokenizing done Mamang!!!
Status data: 8047 valid, 177 empty lists (2.15%)
Proses Stopword Removal done Mamang!!!
Status data: 8047 valid, 177 empty lists (2.15%)
Proses Stemming done Mamang!!!
Status data: 8042 valid, 182 missing/empty (2.21%)
Proses Join Tokens to Text done Mamang!!!

RINGKASAN HASIL PREPROCESSING:
Total data awal: 8224
Total data valid setelah preprocessing: 8042
Total data invalid/kosong: 182 (2.21%)

Analisis data kosong:

Data kosong #17:
Teks asli: ⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐😼⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐💛⭐⭐⭐⭐⭐⭐⭐⭐😼⭐⭐😼⭐🌛⭐⭐😼⭐😼⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐🌟⭐⭐⭐⭐⭐⭐🌟⭐⭐⭐⭐⚡⭐⭐⭐⭐⭐⭐🎊⭐⭐⭐⭐⭐⭐💛⭐⭐⭐⭐⭐⭐⭐⭐⭐🌝⭐⭐⭐⭐🌛⭐⭐⭐🕳️🌜⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐

,score,content,clean_text,case_foldingText,normalized_slangwordsText,tokenized_text,stopword_removedText,stemmed_text,final_text
0,2,"sedih bgt njir, akun aku hilang ud level 40an",sedih bgt njir akun aku hilang ud level an,sedih bgt njir akun aku hilang ud level an,sedih banget aduh akun saya hilang sudah level an,"[sedih, banget, aduh, akun, saya, hilang, suda...","[sedih, aduh, akun, hilang, level]","[sedih, aduh, akun, hilang, level]",sedih aduh akun hilang level
1,1,game bayak buk bintang 1 Update terus buk nya ...,game bayak buk bintang Update terus buk nya ng...,game bayak buk bintang update terus buk nya ng...,game bayak bug bintang update terus bug nya ti...,"[game, bayak, bug, bintang, update, terus, bug...","[bayak, bug, bintang, update, bug, perbaiki, b...","[bayak, bug, bintang, update, bug, baik, bug, ...",bayak bug bintang update bug baik bug damage p...
2,5,baguss gamenya,baguss gamenya,baguss gamenya,bagus gamenya,"[bagus, gamenya]","[bagus, gamenya]","[bagus, gamenya]",bagus gamenya
3,1,ehh min lu klo ngasih tim yg ngotak apaa masa ...,ehh min lu klo ngasih tim yg ngotak apaa masa ...,ehh min lu klo ngasih tim yg ngotak apaa masa ...,ehh admin kamu kalau memberi tim yang masuk ak...,"[ehh, admin, kamu, kalau, memberi, tim, yang, ...","[ehh, admin, tim, masuk, akal, apaa, clash, sq...","[ehh, admin, tim, masuk, akal, apaa, clash, sq...",ehh admin tim masuk akal apaa clash squad elit...
4,5,bagus banget game nya,bagus banget game nya,bagus banget game nya,bagus banget game nya,"[bagus, banget, game, nya]",[bagus],[bagus],bagus


- Dan alhamdulillah hasilnya lancar dan aman ya, tapi masalahnya disini ada missing values nih setelah kita melakukan preprocessing kepada text, kemungkinan ini adalah emoji emoji yang di replace menjadi nilai yang kosong, maka dari itu kita akan menghapusnya saja.